In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import sklearn
import os

In [73]:
data_dir = 'E:\\dataset\\Competition_170323'

In [74]:
train_path = os.path.join(data_dir, 'train.csv')
test_path = os.path.join(data_dir, 'test.csv')

In [75]:
train_raw_data = pd.read_csv(train_path)
test_raw_data = pd.read_csv(test_path)

# 尝试使用每一个属性的均值来填充缺失值

In [76]:
for col_name in train_raw_data.columns:
    if not col_name.endswith('flag'):
        train_raw_data[col_name].fillna(train_raw_data[col_name].mean(), inplace=True)

In [77]:
for col_name in test_raw_data.columns:
    if (col_name != 'ID') and (not col_name.endswith('flag')):
        test_raw_data[col_name].fillna(test_raw_data[col_name].mean(), inplace=True)

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [79]:
train_X = train_raw_data.ix[:, :-1].values
train_y = train_raw_data.ix[:, -1].values

In [80]:
from sklearn import preprocessing

In [81]:
test_X = test_raw_data.ix[:, 1:].values
test_ids = test_raw_data.ix[:, 0].values

In [82]:
train_X = preprocessing.minmax_scale(train_X)
test_X = preprocessing.minmax_scale(test_X)

In [27]:
train_X_pre = train_X[:, 0:55]
test_X_pre = test_X[:, 0:55]

In [83]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

In [84]:
clf = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=-1, random_state=1, verbose=1)

In [85]:
clf.fit(X_train, y_train)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
D:\Anaconda2\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = r'D:\Anaconda2\lib\site-packages\ipykernel\__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

....................................................................

In [68]:
clf.oob_score_

0.50271180731008924

In [69]:
pred_y = clf.predict_proba(X_test)[:, 1]

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    1.7s finished


In [70]:
clf.classes_

array([0, 1], dtype=int64)

In [71]:
from sklearn.metrics import log_loss

In [72]:
log_loss(y_test, pred_y)

0.74872692447571032

In [51]:
pred_y.shape

(63545, 2)

In [55]:
pred_y = pred_y[:, 1]

In [56]:
pred_y.shape

(63545L,)

In [57]:
test_id.shape

(63545,)

In [57]:
result_path = os.path.join(data_dir, 'prediction.csv')

In [58]:
result_file = open(result_path, 'w')

In [59]:
head = '"ID","Target"\n'
result_file.write(head)

In [60]:
for s_id, s_pro in zip(test_ids, pred_y_1):
    line = str(s_id) + ',' + str(s_pro) + '\n'
    result_file.write(line)

In [61]:
result_file.close()